텍스트 전처리의 가장 기본적인 파이프라인를 실습해보는 코드 과제입니다.

과제에 주어진 방식대로 할 필요 없이, 여러분이 원하는 시도를 마음껏 해보셔도 좋습니다
(다른 형태소 분석기를 사용하거나, 불용어를 추가하거나 etc).

## Goal

프로틴 보충제 사이트에서 리뷰를 크롤링해 워드 클라우드 그리기!

파이프라인

1.   정규표현식으로 한글 추리기
2.   띄어쓰기 교정 (PyKospacing)
3.   형태소로 토큰화 (okt)
4.   불용어 제거
5.   원하는 단어 Konlpy 사전에 추가
6.   워드클라우드 그리기


## 1. import, load data

In [ ]:
!pip install konlpy
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
import pandas as pd
import numpy as np
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 1 예습과제 - text preprocessing/protein.csv")

In [ ]:
df.head()

## 2. 정규 표현식으로 한글 단어만 남기기

정규표현식으로 한글단어만 남기고 모두 제외하기

정규표현식이란?

* 특정 규칙이 있는 텍스트 데이터를 정제하는 것
* 정규표현식 모듈 re를 통해 구현 가능

In [ ]:
def extract_hangul(text):
    hangul = re.sub('[^가-힣]', ' ', text) # 한글과 공백 제외하고 지우기
    return hangul

In [ ]:
example = extract_hangul(df['text'][1])
print("전처리 이전: ",df['text'][1])
print("전처리 이후: ",example)

## 3. 띄어쓰기 교정(PyKoSpacing)

PyKoSpacing은 띄어쓰기 교정해주는 패키지

1. re 모듈로 한글외 다른 문자 공백화 -> Py로 공백뭉치들 하나의 공백으로
2. 일반적인 띄어쓰기도 적용

In [ ]:
from pykospacing import Spacing
spacing = Spacing()

In [ ]:
def spacing_text(text):
    spacing_text = spacing(text)
    return spacing_text

In [ ]:
spaced = spacing_text(example)
print("띄어쓰기 전: ",example)
print("띄어쓰기 후: ",spaced)

## 4. 형태소 분석기 (okt)

한국어에서 토큰화 해주는 대표적인 도구 -> konlpy

그중 가장 대표적인 형태소 분석기 -> Okt


*   토큰화란?

주어진 텍스트에서 토큰(token)이라 불리는 단위로 나누는 작업

토큰의 단위가 상황에 따라 다르지만, 보통 의미있는 단위로 토큰을 정의

보통 단어(word) 로 나눈다.



*   형태소란?

의미를 가지는 가장 작은 말의 단위



In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
def extract_morphs(text):
    morphs = okt.morphs(text)
    return morphs # morph 는 형태소를 반환하는 메소드

In [ ]:
morphs_ = extract_morphs(spaced)
print("형태소 분석 전: ",spaced)
print("형태소 분석 후: ",morphs_)

## 5. 불용어 제거

분석에 있어 큰 의미가 없는 '불용어(stopword)'를 지정해줘야 함

불용어를 텍스트 파일로 따로 저장해서 지정

In [ ]:
with open('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 1 예습과제 - text preprocessing/stopword.txt') as f:
    list_file = f.readlines()
stopwords_list = []
for stopword in list_file:
    stopwords = re.sub(r'[\n]', '', stopword) # 줄바꿈, 공백 제거
    stopwords_list.append(stopwords) # 채우기

def remove_stopwords(text):
    remove_stop = [x for x in text if x not in stopwords_list]
    return remove_stop

In [ ]:
remove_stop = remove_stopwords(morphs_)
print("불용어 제거 전: ",morphs_)
print("불용어 제거 후: ",remove_stop)

한글자 단어들은 모두 지우는게 더 나은 경우가 있다. 다만 '맛' '짱' 같은 단어들은 남기기

In [ ]:
def remove_one(text):
    except_list = ['맛', '향', '짱']
    remove_one = [x for x in text if len(x) > 1 or x in except_list] # 중요단어 제외한 한글자 단어 지우기
    return remove_one

In [ ]:
remove_one = remove_one(remove_stop)
print("한글자 단어 제거 전: ",remove_stop)
print("한글자 단어 제거 후: ",remove_one)

모든 형태소가 똑같이 처리되기 때문에 '가성비' 같은 단어가 '가' '성비' 로 나뉘는 경우가 있다.

이때는 konlpy 사전에 직접 단어를 추가 ->
os 모듈을 이용해 코랩 패키지에 저장된 konlpy 폴더에 직접 조작

chdir 로 경로를 이동해주고, makedirs로 임시폴더를 만든 후, 임시폴더에서 단어사전을 수정한 후 원본폴더에 저장해준다.

** FileNotFoundError가 발생하는 경우 아래 directory에서 python 버전이 잘못된 경우일 수 있습니다.

In [ ]:
import os

os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()
os.makedirs('./aaaa')

In [ ]:
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java/aaaa') #임시 폴더로 이동
os.getcwd()

In [ ]:
# 임시폴더에 konlpy 사전 파일의 압축 풀기

!jar xvf ../open-korean-text-2.1.0.jar

압축이 풀렸으니 names.txt 열어보기

In [ ]:
with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/aaaa/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()

In [ ]:
data

우선 세개의 단어 추가, 쓰기모드로 변경해 파일저장

+ 몇단어 추가

In [ ]:
data += '프로틴\n가성비\n밀크티\n초코맛\n딸기맛\n'

with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/aaaa/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)

이제 다시 파일 압축 시키기

* 런타임을 재실행 해야 함

In [ ]:
!jar cvf ../open-korean-text-2.1.0.jar *

In [ ]:
print(okt.nouns("가성비"))

전과정 하나로 통합

In [ ]:
from konlpy.tag import Okt
okt = Okt()

from pykospacing import Spacing
spacing = Spacing()

except_list = ['맛','향','짱']

with open('/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 1 예습과제 - text preprocessing/stopword.txt') as f:
    list_file = f.readlines()

stopwords_list = []
for stopword in list_file:
  stopwords = re.sub('[\n]', '', stopword)
  stopwords_list.append(stopwords)

def review_to_words(raw_review):
  text = re.sub('[^가-힣]', ' ', raw_review)
  text = spacing(text)
  text = okt.morphs(text, stem=True)
  text = [x for x in text if x not in stopwords_list]
  text = [x for x in text if len(x)>1 or x in except_list]
  text = " ".join(text)
  return text

이제 word_list를 만들어 전처리한 단어들을 모아주기

In [ ]:
import tqdm
df_len = df.shape[0]
word_list = []
for i in tqdm.tqdm(range(df_len)):
  word_list.append(review_to_words(df['text'][i]))

##wordcloud 로 시각화

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

%matplotlib inline
def displayWorldCloud(data = None, backgroundcolor = 'white', width = None, height = None):
    wordcloud = WordCloud(font_path = '/content/drive/MyDrive/24-winter KUBIG NLP/WEEK 1 예습과제 - text preprocessing/MALGUN.TTF',
                          background_color = backgroundcolor,
                          width = width,
                          height = height).generate(data)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
displayWorldCloud(data = ' '.join(word_list), width=600, height=400)

## 번외 다른 형태소 분석기 사용

In [ ]:
!pip install pip==20.0.2 pororo fairseq==0.10.2

!pip uninstall torch torchvision
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
from pororo import Pororo

Pororo.available_models("dp")
dp = Pororo(task="dep_parse", lang="ko")

In [ ]:
def extract_morphs(text):
    morphs_po = dp(text)
    return morphs_po # morph 는 형태소를 반환하는 메소드

In [ ]:
morphs_ = extract_morphs(spaced)
print("형태소 분석 전: ",spaced)
print("형태소 분석 후: ",morphs_)